# PPO 
UwU

<p style="font-size: 17px;margin:0">PPO is widely used for its stability. Let's make one!<p>
<p style="font-size: 17px;margin:0">Idea is instead of E[log pi(a | s) * advantage] </p>
<p style="font-size: 17px;margin:0">We use E[new_pi(a|s)/old_pi(a|s) * advantage]</p>

<p style="font-size: 17px">This is the same advantage as we use for actor critic (r + y * V(next state) - V(current state))</p>
<p style="font-size: 17px;margin:0">However we will change up this advantage function by using Generative Advantage Estimation (GAE) instead. This is optional but highly recommended. Its implementation is already filled out for you</p>

In [ ]:
# import everything we need
import gym
import tensorflow as tf
import copy
import numpy as np

### This is where we make the PPO class

In [1]:
class PPO:
    # takes in state dimension, action dimension, learning rate, gamma, and clip value
    def __init__(self, state_dim, action_dim, lr, gamma, clip_val):
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.lr = lr
        self.gamma = gamma
        self.clip_val = clip_val
        
        # Here we will make the placeholders. Make sure to give these placeholders names!
        # Besides the action being of type tf.int32, everything has type tf.float32
        # Make a state placeholder. It should have shape [None, state_dim]
        # YOUR CODE HERE
        
        # Make an action placeholder. It should have shape [None, ] and be of type tf.int32
        # YOUR CODE HERE
        
        # Make a reward placeholder. It should have shape [None, ]
        # YOUR CODE HERE
        
        # Make a advantage placeholder. It should have shape [None, ]
        # YOUR CODE HERE
        
        # Make a next value placeholder. It should have shape [None, ]
        # YOUR CODE HERE
        
        pi_probs, self.v_pred, pi_params = # YOUR CODE HERE build new network
        oldpi_probs, _, oldpi_params = # YOUR CODE HERE build old network
        
        # Use tf.multinomial to select a random action based on our new policy's probability distribution
        # Since this returns a 1 by 1 array we need to use tf.squeeze to force it into a scalar value
        # YOUR CODE HERE
        
        # sets parameters of the old policy to equal to the parameters of the new policy
        self.update_oldpi_op = [oldp.assign(p) for p, oldp in zip(pi_params, oldpi_params)]
        
        # We only need the probability of the action we actually took
        # Create a one hot encoding of our actions (action placeholder)
        # YOUR CODE HERE
        
        # Determine probability of the action of our new policy and old policy
        # Use tf.reduce_sum of axis 1 to compress down our probabilities
        act_prob = # YOUR CODE HERE
        old_act_prob = # YOUR CODE HERE
        
        # Determine critic loss
        with tf.variable_scope('critic_loss'):
            # Our critic loss is still same as our actor critic's critic loss
            # reward + gamma * next value (use the placeholder) - current value
            self.critic_loss = tf.reduce_mean(
                # YOUR CODE HERE
            );
        
        with tf.variable_scope('actor_loss'):
            # Our actor loss is ratio of our new actor's prob and old actor's prob multiplied by advantage
            ratio = # YOUR CODE HERE just calculate ratio for now
            clipped_ratio = # YOUR CODE HERE clip our ratio with tf.clip_by_value
            
            self.actor_loss = tf.reduce_mean(
                # YOUR CODE HERE
            );
        
        with tf.variable_scope('total_loss'):
            self.total_loss = # YOUR CODE HERE find total loss
            optimizer = # YOUR CODE HERE use adam optimizer
            self.train_op = # YOUR CODE HERE minimize total loss (IMPORTANT! var_list = pi_params)
            # We only want to train our new policy. Don't update our old policy!!!!     
    
    # Builds the network
    def _build_network(self, scope):
        with tf.variable_scope(scope):
            # Creates policy network
            with tf.variable_scope('policy'): 
                # Create a 4 layer network (two hidden layers each with 40 neurons and 
                # output layer with number of neurons equal to number of possible actions)
                # Also create a variable named probs equal to the softmax probability of your output layer
                # YOUR CODE HERE
            
            # Create critic network
            with tf.variable_scope('critic'):
                # Create a 4 layer network (two hidden layers each with 40 neurons and 
                # output layer with number of neurons equal to 1)
                # Also create a variable named v_pred equal to your output layer
                # YOUR CODE HERE
        
        # Grab all of the trainable variables of both policy and critic networks within our scope
        # We will need this to update our old policy parameters
        # Look into tf.get_collection. Remeber to use scope!
        params = # YOUR CODE HERE
        return probs, v_pred, params
    
    def get_action(self, sess, state):
        return # return random action

    def get_value(self, sess, state):
        return # return value of state

    def update_params(self, sess):
        # update parameters of old network

    def train(self, sess, states, actions, rewards, advantages, next_vs):
        feed_dict = # Create dictionary of placeholders : values
        total_loss, actor_loss, critic_loss, _ = # Get total loss, actor loss, critic loss, and run train op
        return total_loss, actor_loss, critic_loss

SyntaxError: invalid syntax (<ipython-input-1-204e3ee7b4cb>, line 27)

### GAE

In [ ]:
def get_gaes(gamma, rewards, v_preds, v_preds_next):
    # Code is in GAE file.
    # Huge props if you make it yourself

### Main

In [ ]:
def main():
    env = gym.make('Cartpole-v0')
    env.seed(0)
    gamma = 0.99
    
    with tf.Session() as sess:
        # Create agent and initialize variables
        # YOUR CODE HERE
        
        for iteration in range(100000):
            states = []
            actions = []
            v_preds = []
            rewards = []
            total_reward = 0
            
            obs = env.reset()
            while True:
                action = # YOUR CODE HERE get agent's action. Use [np.newaxis, :] to add a dimension to state
                value = # YOUR CODE HERE get value of the state. 
                # The value will be a 1 by 1 list but we need the scalar value
                
                new_obs, reward, done, info = env.step(action)
                total_reward += reward
                
                tates.append(obs)
                actions.append(action)
                rewards.append(reward)
                v_preds.append(value)

                obs = new_obs

                if done:
                    # The value list goes from the first state to last state
                    # However we want the values from the second state to last state + [0]
                    # We need values of next states not current states
                    v_preds_next =  # YOUR CODE HERE
                    print(sum(rewards))
                    break
                    
            gaes = np.array(get_gaes(gamma, rewards, v_preds, v_preds_next))
            gaes = (gaes - gaes.mean()) / gaes.std() # normalize advantages for stability
            
            states = np.array(states)
            actions = np.array(actions).astype(dtype=np.int32)
            rewards = np.array(rewards).astype(dtype=np.float32)
            v_preds_next = np.array(v_preds_next).astype(dtype=np.float32)
            
            # Update our old network's parameters
            # YOUR CODE HERE
            
            inp = [states, actions, rewards, v_preds_next, gaes]
            
            # Idea is we train it four times while clipping values
            for epoch in range(4):
                sample_indices = np.random.randint(low=0, high=states.shape[0], size=64)  # indices are in [low, high)
                sampled_inp = [np.take(a=a, indices=sample_indices, axis=0) for a in inp]  # sample training data
                agent.train(sess, sampled_inp[0], sampled_inp[1], sampled_inp[2], 
                            sampled_inp[3], sampled_inp[4])

                
if __name__ == "__main__":
    main()